In [2]:
!pip install datasets
!pip install evaluate

from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict, Dataset
import matplotlib.pyplot as plt
import pandas as pd
import torch
import evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [4]:
# Convert dataset splits to pandas DataFrame
train_df = dataset["train"].to_pandas()
validation_df = dataset["validation"].to_pandas() if "validation" in dataset else None
test_df = dataset["test"].to_pandas() if "test" in dataset else None

def get_subset(df):
    subset_df = df.groupby("label").head(50)
    subset_df = subset_df.reset_index(drop=True)
    return subset_df

subset_train_df = get_subset(train_df)
subset_validation_df = get_subset(validation_df) if validation_df is not None else None
subset_test_df = get_subset(test_df) if test_df is not None else None

# Convert the subset DataFrames back to DatasetDict object
subset_train_dataset = Dataset.from_pandas(subset_train_df)
subset_validation_dataset = Dataset.from_pandas(subset_validation_df) if subset_validation_df is not None else None
subset_test_dataset = Dataset.from_pandas(subset_test_df) if subset_test_df is not None else None

subset_dataset_dict = DatasetDict({
    "train": subset_train_dataset,
    "validation": subset_validation_dataset,
    "test": subset_test_dataset
})


In [5]:
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

import evaluate

def preprocess_function(examples):
    """
    Preprocesses text data by tokenizing it using the provided tokenizer.

    Args:
        examples (dict): A dictionary containing text data to be preprocessed.

    Returns:
        dict: A dictionary containing the preprocessed data (tokenized text).
    """

    return tokenizer(examples['text'], padding=True, truncation=True)


# Load the pre-trained model checkpoint (replace "new_model_name" with your chosen model)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

encoded_Data = subset_dataset_dict.map(preprocess_function,batched=True, batch_size=5568)
encoded_Data = encoded_Data.remove_columns(['text'])
# Split the dataset into train, validation, and test sets (if available)
train_df = encoded_Data["train"]
validation_df = encoded_Data["validation"]
test_df = encoded_Data["test"]




tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
#!pip uninstall accelerate -y
!pip install accelerate>=0.21.0 -U
#!pip install transformers[torch]

# Get the number of labels from the dataset (replace with actual number of labels)
num_labels = len(set(train_df["label"]))

# Define label-to-ID and ID-to-label dictionaries (adapt based on your labels)
label_id = {"negative": 0, "positive": 1}  # Example mapping
id_label = {0: "negative", 1: "positive"}  # Example mapping

# Load the model for sequence classification with the appropriate number of labels
model = (
    AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=num_labels, id2label=label_id, label2id=id_label
    )
    .to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
)

# Define the evaluation metric (replace with a suitable metric from evaluate or custom implementation)
metric = evaluate.load("accuracy")

# Define training arguments (adjust parameters as needed)
batch_size = 16
learning_rate = 2e-5
num_epochs = 10

training_args = TrainingArguments(
    output_dir="fine-tuned-model",
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_steps=len(train_df) // batch_size,
)




model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

# Create a Trainer instance to handle training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_df,
    eval_dataset=validation_df,
    compute_metrics=compute_metrics,
)

# Train the model
history = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.705700,0.689504,0.500000,0.333333
2,0.682300,0.675287,0.660000,0.631076
3,0.674500,0.658186,0.690000,0.665624
4,0.641800,0.630888,0.790000,0.785166
5,0.583300,0.590253,0.840000,0.838969
6,0.486800,0.546571,0.860000,0.859944
7,0.426800,0.508269,0.880000,0.879952
8,0.390000,0.482501,0.840000,0.839936
9,0.320100,0.468530,0.840000,0.839744
10,0.295600,0.462381,0.850000,0.849865


In [8]:
# Evaluate the model on the test set (if available)
if test_df is not None:
    test_results = trainer.evaluate(eval_dataset=test_df)
test_results

{'eval_loss': 0.6287512183189392,
 'eval_accuracy': 0.63,
 'eval_f1': 0.629666700030027,
 'eval_runtime': 9.638,
 'eval_samples_per_second': 10.376,
 'eval_steps_per_second': 0.726,
 'epoch': 10.0}